In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow import keras

%load_ext autoreload
%autoreload 2

import main

In [103]:
labels = pd.Series(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

train_path = '../blobs/notMNIST_large'
test_path = '../blobs/notMNIST_small'

Loading file names

In [104]:
from sklearn.model_selection import train_test_split

x_train_files = []
y_train = []
x_val_files = []
y_val = []
x_test_files = []
y_test = []
for label, index in zip(labels, labels.index):    
    train_dir = os.path.join(train_path, label)
    files = pd.Series(os.path.join(train_dir, name) for name in os.listdir(train_dir))
    # NOTE: reducing the number of samples for faster processing
    files = files.sample(len(files) // 10)
    x_train_files.extend(files)
    y_train.extend(index for _ in range(len(files)))
    
    # val and test must come from the same distribution
    test_dir = os.path.join(test_path, label)
    test_files = [os.path.join(test_dir, name) for name in os.listdir(test_dir)]
    val, test = train_test_split(test_files, test_size=0.5)
    x_val_files.extend(val)
    y_val.extend(index for _ in range(len(val)))
    x_test_files.extend(test)
    y_test.extend(index for _ in range(len(test)))
    
print('# of train files:', len(x_train_files))
assert(len(x_train_files) == len(y_train))
print('# of val files:', len(x_val_files))
assert(len(x_val_files) == len(y_val))
print('# of test files:', len(x_test_files))
assert(len(x_test_files) == len(y_test))

# of train files: 51696
# of val files: 9360
# of test files: 9364


Shuffling file names to avoid labels being consecutive

In [4]:
main.shuffle_in_unison(x_train_files, y_train)
main.shuffle_in_unison(x_val_files, y_val)
main.shuffle_in_unison(x_test_files, y_test)

In [105]:
x_test = np.array([main.load_image(file) for file in x_test_files])
y_test = np.array(y_test)
x_test = x_test / 255.0
print('Test set:', x_test.shape)

Test set: (9364, 28, 28)


In [7]:
x_train = np.array([main.load_image(file) for file in x_train_files])
y_train= np.array(y_train)
x_train = x_train / 255.0
print(x_train.shape)

(51696, 28, 28)


In [106]:
x_val = np.array([main.load_image(file) for file in x_val_files])
y_val= np.array(y_val)
x_val = x_val / 255.0
print('Validation set:', x_val.shape)

Validation set: (9360, 28, 28)


In [107]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(150, activation='relu'),
    keras.layers.Dense(50, activation='relu'),
    keras.layers.Dense(10)
])
model.compile(
    optimizer='SGD', 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

In [89]:
model.fit(x_train, y_train, epochs=30)

Epoch 1/30
51696/51696 [==============================] - 3s 56us/sample - loss: 0.8119 - acc: 0.7819
Epoch 2/30
51696/51696 [==============================] - 3s 53us/sample - loss: 0.6114 - acc: 0.8279
Epoch 3/30
51696/51696 [==============================] - 3s 58us/sample - loss: 0.5632 - acc: 0.8377
Epoch 4/30
51696/51696 [==============================] - 3s 56us/sample - loss: 0.5315 - acc: 0.8464
Epoch 5/30
51696/51696 [==============================] - 3s 54us/sample - loss: 0.5068 - acc: 0.8525
Epoch 6/30
51696/51696 [==============================] - 3s 53us/sample - loss: 0.4857 - acc: 0.8574
Epoch 7/30
51696/51696 [==============================] - 3s 55us/sample - loss: 0.4682 - acc: 0.8628
Epoch 8/30
51696/51696 [==============================] - 3s 58us/sample - loss: 0.4522 - acc: 0.8661
Epoch 9/30
51696/51696 [==============================] - 3s 59us/sample - loss: 0.4364 - acc: 0.8710
Epoch 10/30
51696/51696 [==============================] - 3s 55us/sample - loss: 

In [90]:
print('Test prediction:')
score, accuracy = model.evaluate(x_test, y_test)

Test prediction:
9364/9364 [==============================] - 0s 42us/sample - loss: 0.2560 - acc: 0.9244


Logistic regression was at about 88% accuracy on 50k train samples. Here we got **92.5%** without any fine-tuning, although the model overits

Let's try **regularization**:

In [91]:
reg_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(150, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(50, activation='relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(10, kernel_regularizer=keras.regularizers.l2(0.0001))
])
reg_model.compile(
    optimizer='sgd', 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

In [92]:
reg_history = reg_model.fit(x_train, y_train, epochs=30)

Epoch 1/30
51696/51696 [==============================] - 4s 73us/sample - loss: 0.9359 - acc: 0.7376
Epoch 2/30
51696/51696 [==============================] - 3s 67us/sample - loss: 0.6760 - acc: 0.8160
Epoch 3/30
51696/51696 [==============================] - 3s 64us/sample - loss: 0.6259 - acc: 0.8265
Epoch 4/30
51696/51696 [==============================] - 4s 70us/sample - loss: 0.5946 - acc: 0.8348
Epoch 5/30
51696/51696 [==============================] - 4s 70us/sample - loss: 0.5759 - acc: 0.8405
Epoch 6/30
51696/51696 [==============================] - 5s 99us/sample - loss: 0.5601 - acc: 0.8442
Epoch 7/30
51696/51696 [==============================] - 4s 86us/sample - loss: 0.5441 - acc: 0.8490
Epoch 8/30
51696/51696 [==============================] - 4s 69us/sample - loss: 0.5307 - acc: 0.8525
Epoch 9/30
51696/51696 [==============================] - 4s 77us/sample - loss: 0.5187 - acc: 0.85592s - 
Epoch 10/30
51696/51696 [==============================] - 4s 79us/sample - l

In [112]:
# Using this to tune the drop rate and L2 parameters
print('Validation prediction:')
score, accuracy = reg_model.evaluate(x_val, y_val)

Validation prediction:
9360/9360 [==============================] - 0s 36us/sample - loss: 0.2586 - acc: 0.9363


In [113]:
print('Test prediction:')
score, accuracy = reg_model.evaluate(x_test, y_test)

Test prediction:
9364/9364 [==============================] - 0s 36us/sample - loss: 0.2649 - acc: 0.9356


How about adding **learning rate decay**:

Using regularization increased the accuracy, though not by much, about **1%**. But now we can safely run even more epochs without the fear of overfitting.

In [114]:
adaptive_model = keras.models.clone_model(reg_model)
# We can also try using other algorithms instead of decay in SGD
adaptive_model.compile(
    optimizer=keras.optimizers.SGD(decay=1e-5), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

In [100]:
adaptive_history = adaptive_model.fit(x_train, y_train, epochs=30)

Epoch 1/30
51696/51696 [==============================] - 4s 77us/sample - loss: 0.9373 - acc: 0.7415
Epoch 2/30
51696/51696 [==============================] - 4s 71us/sample - loss: 0.6767 - acc: 0.8136
Epoch 3/30
51696/51696 [==============================] - 3s 65us/sample - loss: 0.6242 - acc: 0.8270
Epoch 4/30
51696/51696 [==============================] - 4s 70us/sample - loss: 0.5948 - acc: 0.8345
Epoch 5/30
51696/51696 [==============================] - 4s 71us/sample - loss: 0.5724 - acc: 0.8418
Epoch 6/30
51696/51696 [==============================] - 4s 73us/sample - loss: 0.5549 - acc: 0.8461
Epoch 7/30
51696/51696 [==============================] - 4s 74us/sample - loss: 0.5421 - acc: 0.8498
Epoch 8/30
51696/51696 [==============================] - 4s 75us/sample - loss: 0.5285 - acc: 0.8526
Epoch 9/30
51696/51696 [==============================] - 4s 74us/sample - loss: 0.5153 - acc: 0.8570
Epoch 10/30
51696/51696 [==============================] - 4s 74us/sample - loss: 

In [110]:
print('Validation prediction:')
score, accuracy = adaptive_model.evaluate(x_val, y_val)

Validation prediction:
9360/9360 [==============================] - 0s 48us/sample - loss: 0.2616 - acc: 0.9347


In [111]:
print('Test prediction:')
score, accuracy = adaptive_model.evaluate(x_test, y_test)

Test prediction:
9364/9364 [==============================] - 0s 36us/sample - loss: 0.2665 - acc: 0.9330


Rate decay doesn't seem to help much here, results a bit worse. Could try more epochs

What else we can **try**:

- load all the available train data
- teach for more epochs
- try other optimization algorithms, e.g. Adam
- play with the number of layers and hidden neurons using validation set
- try one-hot output encoding
- change activation functions (unlikely)